In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [3]:
# os.chdir('..')
base_dir = os.getcwd()

In [4]:
img_dir = os.path.join(base_dir, 'img')
models_dir = os.path.join(base_dir, 'models')
# img_dir = '/content/drive/MyDrive/JetsonNano/images'
# models_dir = '/content/drive/MyDrive/JetsonNano/model'

In [5]:
base_dir

'/home/claudic/JetsonNano/eye_close_project'

In [6]:
# Define the data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the same size as expected by ResNet
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

dataset = datasets.ImageFolder(img_dir, transform=transform)

# Split the dataset into train, test, and validation sets
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])

# Create data loaders for train, test, and validation sets
batch_size = 64  # Replace with your desired batch size

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

print(f"Train dataset size: {len(train_loader.dataset)}")
print(f"Test dataset size: {len(test_loader.dataset)}")
print(f"Validation dataset size: {len(val_loader.dataset)}")

Train dataset size: 8000
Test dataset size: 1000
Validation dataset size: 1000


In [7]:
dataset.class_to_idx

{'close': 0, 'open': 1}

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet-34 model
# model = models.resnet34(pretrained=True)
model = models.resnet18(pretrained=True)

# Modify the last fully connected layer for the desired number of classes
num_classes = 2  # Replace with the actual number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/claudic/anaconda3/envs/ecg-classification/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/claudic/anaconda3/envs/ecg-classification/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/claudic/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


In [9]:
print(device)

cuda


In [10]:
# Train the model
num_epochs = 20  # Replace with the desired number of training epochs
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:  # Replace train_loader with your data loader
        inputs = inputs.to(device)  # Move inputs to the device
        labels = labels.to(device)  # Move labels to the device

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")



Epoch 1/20, Loss: 0.1931429751366377
Epoch 2/20, Loss: 0.05602213026583195
Epoch 3/20, Loss: 0.035386771477758885
Epoch 4/20, Loss: 0.025027242696844042
Epoch 5/20, Loss: 0.019802252360619603
Epoch 6/20, Loss: 0.010832715459167957
Epoch 7/20, Loss: 0.00852872961666435
Epoch 8/20, Loss: 0.005358825159259141
Epoch 9/20, Loss: 0.004706163699505851
Epoch 10/20, Loss: 0.005521663679741323
Epoch 11/20, Loss: 0.0033315168782137336
Epoch 12/20, Loss: 0.004250545783434063
Epoch 13/20, Loss: 0.0018686118660261855
Epoch 14/20, Loss: 0.002219858916127123
Epoch 15/20, Loss: 0.0013830276886001229
Epoch 16/20, Loss: 0.001411224022100214
Epoch 17/20, Loss: 0.0008676790577592328
Epoch 18/20, Loss: 0.0010012135698925703
Epoch 19/20, Loss: 0.0010360450636944733
Epoch 20/20, Loss: 0.0009503972740494646


In [11]:
# Save the trained model
torch.save(model.state_dict(), f'{models_dir}/resnet18-e_20-d_10k.pth')

In [12]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model.eval()  # Set the model to evaluation mode

y_true = []
y_pred = []

with torch.no_grad():  # Disable gradient calculation for validation
    for inputs, labels in val_loader:
        inputs = inputs.to(device)  # Move inputs to the device
        labels = labels.to(device)  # Move labels to the device

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the predicted labels

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
confusion_mat = confusion_matrix(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred)

print(f"Validation Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(classification_rep)

Validation Accuracy: 0.984
Confusion Matrix:
[[500   9]
 [  7 484]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       509
           1       0.98      0.99      0.98       491

    accuracy                           0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000

